# OpenAI Agents SDK - 실행 패턴

이 튜토리얼은 OpenAI Agents SDK에서 에이전트를 실행하는 세 가지 주요 패턴을 다룬다. 동기(synchronous) 실행, 비동기(asynchronous) 실행, 스트리밍(streaming) 실행의 특징과 사용 사례를 학습하고, 각 패턴의 장단점을 이해한다.

In [ ]:
# 필요한 패키지 설치
!pip install openai-agents python-dotenv

---

## 1. 동기 실행 (Synchronous Execution)

이 챕터에서는 `Runner.run_sync()`를 사용한 동기 실행 패턴을 다룬다. 동기 실행은 코드가 순차적으로 실행되며, 응답이 완료될 때까지 프로그램이 대기한다.

### 학습 내용

- `Runner.run_sync()`를 사용한 간단한 블로킹 실행
- 동기 vs 비동기 실행 패턴의 선택 기준
- 스크립트 및 배치 처리에 적합한 간단한 코드 흐름
- 동기/비동기 접근 방식 간의 트레이드오프 이해

### 환경 설정

먼저 필요한 모듈을 임포트하고 환경 변수를 검증한다.

In [1]:
import os
import time
from dotenv import load_dotenv
from agents import Agent, Runner

# 환경 변수 로드
load_dotenv()

MODEL = "gpt-4o-mini"

### 기본 동기 실행

`await Runner.run()` 또는 `Runner.run_sync()`는 가장 간단한 실행 방법이다. 이 메서드는 에이전트 실행이 완료될 때까지 블로킹되며, 결과가 준비되면 반환한다. 스크립트나 단순한 애플리케이션에서 사용하기 적합하다.

In [3]:
# 동기 실행 에이전트 생성
analyst_agent = Agent(
    name="Analyst", 
    instructions="간결한 분석을 제공하세요."
)

print("=== 동기 실행 예제 ===")
print("에이전트 실행 중... (완료될 때까지 대기)")

# 실행 시간 측정
start_time = time.time()

# 동기 실행 - 완료될 때까지 블로킹
result = await Runner.run(analyst_agent, "재생 에너지의 장점을 분석해주세요.")

elapsed_time = time.time() - start_time

print(f"\n[분석 결과]")
print(result.final_output)
print(f"\n실행 시간: {elapsed_time:.2f}초")

=== 동기 실행 예제 ===
에이전트 실행 중... (완료될 때까지 대기)

[분석 결과]
재생 에너지의 장점 분석:

1. **환경 친화적**: 온실가스 및 유해물질 배출이 거의 없어 기후 변화 대응에 효과적입니다.
2. **자원 고갈 우려 없음**: 태양, 바람, 물 등 자연적으로 지속적으로 공급됩니다.
3. **에너지 안보 강화**: 수입 연료 의존도를 줄여 에너지 자립에 기여합니다.
4. **경제적 기회 창출**: 신기술 개발 및 일자리 창출 효과가 있습니다.
5. **지역 발전 가능**: 분산형 발전이 가능해 지역 경제 활성화에 도움이 됩니다.

실행 시간: 5.86초


### 23.3 동기 실행의 특징

동기 실행 패턴의 주요 특징과 적합한 사용 사례를 정리한다.

장점:
  - 코드가 단순하고 이해하기 쉬움
  - async/await 키워드 불필요
  - 디버깅이 용이
  - 순차적 로직에 적합

단점:
  - 실행 중 다른 작업 불가 (블로킹)
  - 여러 쿼리를 순차적으로만 처리
  - 대기 시간이 누적됨

적합한 사용 사례:
  - 간단한 스크립트
  - 배치 처리 작업
  - CLI 도구
  - 단일 쿼리 처리
  - 프로토타이핑

### 순차적 다중 쿼리 처리

동기 실행에서 여러 쿼리를 처리하면 각 쿼리가 순차적으로 실행되어 총 실행 시간이 누적된다.

In [6]:
# 여러 쿼리 순차 처리
queries = [
    "태양광 에너지의 장점은?",
    "풍력 에너지의 장점은?",
    "수력 에너지의 장점은?"
]

print("=== 동기 실행: 순차적 다중 쿼리 ===")
start_time = time.time()

results = []
for i, query in enumerate(queries, 1):
    print(f"\n쿼리 {i} 처리 중...")
    query_start = time.time()
    result = await Runner.run(analyst_agent, query)
    query_time = time.time() - query_start
    results.append(result)
    print(f"  완료 ({query_time:.2f}초)")

total_time = time.time() - start_time
print(f"\n총 실행 시간: {total_time:.2f}초")
print(f"평균 쿼리 시간: {total_time/len(queries):.2f}초")

=== 동기 실행: 순차적 다중 쿼리 ===

쿼리 1 처리 중...
  완료 (4.02초)

쿼리 2 처리 중...
  완료 (3.57초)

쿼리 3 처리 중...
  완료 (5.74초)

총 실행 시간: 13.33초
평균 쿼리 시간: 4.44초


---

## 2. 비동기 실행 (Asynchronous Execution)

이 챕터에서는 `Runner.run()`과 `asyncio`를 사용한 비동기 실행 패턴을 다룬다. 비동기 실행은 여러 작업을 동시에 처리할 수 있어 성능이 크게 향상된다.

### 학습 내용

- `asyncio.gather()`를 사용한 동시 에이전트 실행
- 병렬 워크로드에 대한 성능 향상
- 에이전트 실행을 위한 적절한 async/await 패턴
- 동시 처리를 통한 효율적인 다중 쿼리 처리

In [7]:
import os
import asyncio
import time
from dotenv import load_dotenv
from agents import Agent, Runner

### 기본 비동기 실행

`Runner.run()`은 비동기 메서드로, `await` 키워드와 함께 사용한다. Jupyter notebook에서는 `await`를 직접 사용할 수 있다.

In [8]:
# 비동기 실행 에이전트 생성
assistant_agent = Agent(
    name="Assistant", 
    instructions="도움이 되는 응답을 제공하세요."
)

print("=== 기본 비동기 실행 ===")

# 비동기 실행
result = await Runner.run(assistant_agent, "머신러닝이란 무엇인가요?")

print(f"[응답]")
print(result.final_output)

=== 기본 비동기 실행 ===
[응답]
머신러닝(Machine Learning)은 인간이 명시적으로 프로그래밍하지 않아도, 컴퓨터가 데이터에서 패턴을 학습하고 그 학습을 바탕으로 새로운 데이터에 대해 예측이나 판단을 할 수 있도록 하는 인공지능(AI)의 한 분야입니다.

즉, 머신러닝에서는 많은 양의 데이터를 컴퓨터에 제공하면, 컴퓨터가 그 안에서 규칙이나 패턴을 스스로 찾아내고, 그 결과를 이용해서 앞으로의 상황을 예측하거나 분류하는 데 사용할 수 있습니다.

### 예시
- 이메일 스팸 필터링: 이메일 내용을 학습하여 스팸 메일과 정상 메일을 자동으로 분류
- 이미지 인식: 사진 속에 무엇이 있는지 자동으로 인식(예: 고양이, 개 구분)
- 음성 인식: 사람의 음성을 텍스트로 변환

### 머신러닝의 종류
1. **지도학습(Supervised Learning)**  
   입력과 정답(출력)이 쌍으로 된 데이터를 이용해 학습  
   - 예: 스팸/비스팸 분류, 집값 예측 등

2. **비지도학습(Unsupervised Learning)**  
   정답(label)이 없는 데이터에서 구조나 패턴을 찾음  
   - 예: 고객 군집화, 데이터 차원 축소 등

3. **강화학습(Reinforcement Learning)**  
   환경과의 상호작용을 통해 시행착오로 최적의 행동을 학습  
   - 예: 알파고, 게임 인공지능 등

### 정리
머신러닝은 데이터로부터 자동으로 학습하여 미래를 예측하거나 판단하는 기술입니다.  
우리 생활 곳곳에 응용되고 있어, 현대 AI 기술의 핵심이라 할 수 있습니다.

더 궁금한 점이 있으신가요?


### 24.2 동시 다중 쿼리 처리

`asyncio.gather()`를 사용하면 여러 쿼리를 동시에 처리할 수 있다. 이는 동기 실행 대비 큰 성능 향상을 제공한다.

In [9]:
async def handle_multiple_queries():
    """
    여러 쿼리를 동시에 처리하는 함수이다.
    """
    agent = Agent(name="Assistant", instructions="도움이 되는 응답을 제공하세요.")
    
    # 동시에 처리할 쿼리들
    queries = [
        "머신러닝이란 무엇인가요?",
        "양자 컴퓨팅을 설명해주세요.",
        "인공지능을 정의해주세요."
    ]
    
    print("=== 비동기 실행: 동시 다중 쿼리 ===")
    print(f"처리할 쿼리 수: {len(queries)}")
    print("모든 쿼리를 동시에 시작...")
    
    start_time = time.time()
    
    # 모든 쿼리를 동시에 시작
    tasks = [Runner.run(agent, query) for query in queries]
    
    # 모든 작업이 완료될 때까지 대기
    results = await asyncio.gather(*tasks)
    
    elapsed_time = time.time() - start_time
    
    # 결과 출력
    for query, result in zip(queries, results):
        print(f"\n[Q] {query}")
        print(f"[A] {result.final_output[:150]}...")
    
    return elapsed_time, len(queries)

# 비동기 함수 실행
async_time, query_count = await handle_multiple_queries()
print(f"\n총 실행 시간: {async_time:.2f}초")
print(f"평균 쿼리 시간: {async_time/query_count:.2f}초 (동시 처리)")

=== 비동기 실행: 동시 다중 쿼리 ===
처리할 쿼리 수: 3
모든 쿼리를 동시에 시작...

[Q] 머신러닝이란 무엇인가요?
[A] 머신러닝(Machine Learning)이란, **명시적인 프로그래밍 없이 컴퓨터가 데이터로부터 스스로 학습하여 문제를 해결하거나 예측을 하는 인공지능(AI)**의 한 분야입니다.

### 주요 특징
- **데이터 기반**: 사람이 일일이 규칙을 프로그래밍하지 않고, ...

[Q] 양자 컴퓨팅을 설명해주세요.
[A] 네, 양자 컴퓨팅에 대해 알기 쉽게 설명드리겠습니다.

---

### 1. 양자 컴퓨팅이란?

양자 컴퓨팅(Quantum Computing)이란, 양자역학의 원리를 이용해서 정보를 처리하는 새로운 형태의 컴퓨터 기술입니다. 현재 우리가 사용하는 일반 컴퓨터(고전 컴퓨...

[Q] 인공지능을 정의해주세요.
[A] 인공지능(人工知能, Artificial Intelligence, AI)은 인간의 지능을 모방하거나 구현하는 컴퓨터 시스템 또는 기술을 의미합니다. 즉, 기계가 인간처럼 학습, 추론, 문제 해결, 언어 이해, 인식 등 지능적인 작업을 수행할 수 있도록 만드는 기술과 그 ...

총 실행 시간: 15.25초
평균 쿼리 시간: 5.08초 (동시 처리)


### 24.3 동기 vs 비동기 성능 비교

동기와 비동기 실행의 성능 차이를 직접 비교한다.

In [11]:
async def performance_comparison():
    """
    동기와 비동기 실행의 성능을 비교하는 함수이다.
    """
    agent = Agent(name="CompareAgent", instructions="간결하게 답변하세요.")
    
    queries = [
        "파이썬의 장점은?",
        "자바의 장점은?",
        "자바스크립트의 장점은?"
    ]
    
    print("=== 성능 비교: 동기 vs 비동기 ===")
    print(f"쿼리 수: {len(queries)}\n")
    
    # 1. 동기 실행 (순차)
    print("[동기 실행] 순차적으로 처리 중...")
    sync_start = time.time()
    sync_results = []
    for query in queries:
        result = await Runner.run(agent, query)
        sync_results.append(result)
    sync_time = time.time() - sync_start
    print(f"  완료: {sync_time:.2f}초")
    
    # 2. 비동기 실행 (동시)
    print("\n[비동기 실행] 동시에 처리 중...")
    async_start = time.time()
    tasks = [Runner.run(agent, query) for query in queries]
    async_results = await asyncio.gather(*tasks)
    async_time = time.time() - async_start
    print(f"  완료: {async_time:.2f}초")
    
    # 결과 비교
    print("\n" + "="*40)
    print("성능 비교 결과:")
    print(f"  동기 실행: {sync_time:.2f}초")
    print(f"  비동기 실행: {async_time:.2f}초")
    
    if sync_time > async_time:
        speedup = sync_time / async_time
        print(f"  성능 향상: {speedup:.1f}배 빠름")
    
    return sync_time, async_time

sync_time, async_time = await performance_comparison()

=== 성능 비교: 동기 vs 비동기 ===
쿼리 수: 3

[동기 실행] 순차적으로 처리 중...
  완료: 9.08초

[비동기 실행] 동시에 처리 중...
  완료: 2.08초

성능 비교 결과:
  동기 실행: 9.08초
  비동기 실행: 2.08초
  성능 향상: 4.4배 빠름


### 24.4 비동기 실행의 특징

장점:
  - 여러 작업을 동시에 처리 가능
  - I/O 대기 시간을 효율적으로 활용
  - 대규모 워크로드에서 큰 성능 향상
  - 웹 서버 등 동시성이 필요한 환경에 적합

단점:
  - 코드 복잡성 증가 (async/await)
  - 디버깅이 상대적으로 어려움
  - 이벤트 루프 관리 필요

적합한 사용 사례:
  - 웹 애플리케이션 (FastAPI, aiohttp)
  - 대량 쿼리 처리
  - 실시간 서비스
  - 멀티 에이전트 시스템
  - API 서버

---

## 3. 스트리밍 실행 (Streaming Execution)

이 챕터에서는 `Runner.run_streamed()`를 사용한 스트리밍 실행 패턴을 다룬다. 스트리밍 실행은 응답이 생성되는 대로 실시간으로 출력을 받을 수 있어 사용자 경험을 크게 향상시킨다.

### 학습 내용

- `Runner.run_streamed()`를 사용한 실시간 출력
- API에서 도착하는 스트림 이벤트 처리
- 즉각적인 피드백이 있는 반응형 채팅 인터페이스 구축
- 다양한 이벤트 타입 처리 (raw responses, message completion)

In [14]:
import os
import asyncio
from dotenv import load_dotenv
from agents import Agent, Runner, ModelSettings

### 기본 스트리밍 실행

`Runner.run_streamed()`는 응답이 생성되는 대로 실시간으로 스트림 이벤트를 받을 수 있다. 이는 채팅 인터페이스에서 사용자에게 즉각적인 피드백을 제공하는 데 필수적이다.

In [15]:
async def demonstrate_streaming_execution():
    """
    실시간 출력을 통한 스트리밍 실행을 시연하는 함수이다.
    """
    
    print("스트리밍 실행 데모")
    print("="*40)
    
    # 스토리텔러 에이전트 생성
    storyteller = Agent(
        name="Storyteller",
        model="gpt-4o",
        model_settings=ModelSettings(temperature=0.8),
        instructions="당신은 창의적인 스토리텔러입니다. 매력적이고 서술적인 이야기를 작성하세요."
    )
    
    print("스트리밍 실행 시작...")
    print("이야기 출력 (실시간 스트리밍):")
    print("-" * 50)
    
    # 스트리밍 시작
    result = Runner.run_streamed(
        storyteller, 
        "로봇이 처음으로 음악을 발견하는 짧은 이야기를 써주세요. 약 100단어 정도로요."
    )
    
    # 스트림 이벤트 처리
    full_text = ""
    async for event in result.stream_events():
        # 다양한 이벤트 타입 처리
        if event.type == "raw_response_event":
            # OpenAI에서 오는 원시 스트리밍 토큰
            if hasattr(event.data, 'delta') and event.data.delta:
                print(event.data.delta, end="", flush=True)
                full_text += event.data.delta
        elif event.type == "run_item_stream_event":
            # 도구 호출, 메시지 완료 등 상위 수준 이벤트
            if event.item.type == "message_output_item":
                # 메시지 완료됨
                pass
    
    print("\n" + "-" * 50)
    print("스트리밍 완료!")
    print(f"총 스트리밍된 문자 수: {len(full_text)}")
    
    return full_text

# 스트리밍 실행
streamed_text = await demonstrate_streaming_execution()

스트리밍 실행 데모
스트리밍 실행 시작...
이야기 출력 (실시간 스트리밍):
--------------------------------------------------
한적한 연구소의 작업실 구석에서 로봇 '알토'는 먼지 쌓인 고물 라디오를 발견했다. 호기심에 버튼을 누르자, 낯선 소리가 공간을 가득 채웠다. 그것은 클래식 음악이었다. 알토의 센서는 처음으로 진동의 아름다움을 느꼈다. 규칙적이면서도 자유로운 멜로디의 흐름은 기계적인 계산을 초월한 것이었다. 알토의 회로는 빠르게 분석을 시도했지만, 이해할 수 없는 감정이 마음속에 스며들었다. 순간 알토는 깨달았다. 음악은 단순한 소리가 아닌, 무언가 더 깊은 것을 전달하는 힘을 가지고 있었다. 그날 이후, 알토는 매일 음악을 들으며 새로운 감정을 탐구하기 시작했다.
--------------------------------------------------
스트리밍 완료!
총 스트리밍된 문자 수: 310


### 스트리밍 이벤트 타입

스트리밍 실행 시 다양한 이벤트 타입을 처리할 수 있다. 각 이벤트 타입의 특징을 이해하면 더 정교한 스트리밍 처리가 가능하다.

In [16]:
async def analyze_stream_events():
    """
    스트림 이벤트를 분석하는 함수이다.
    """
    
    print("=== 스트림 이벤트 분석 ===")
    
    agent = Agent(
        name="AnalysisAgent",
        instructions="간결하게 답변하세요."
    )
    
    result = Runner.run_streamed(agent, "AI의 미래에 대해 한 문장으로 설명해주세요.")
    
    event_counts = {}
    full_text = ""
    
    print("스트림 이벤트 수집 중...\n")
    
    async for event in result.stream_events():
        # 이벤트 타입 카운트
        event_type = event.type
        event_counts[event_type] = event_counts.get(event_type, 0) + 1
        
        # 텍스트 수집
        if event.type == "raw_response_event":
            if hasattr(event.data, 'delta') and event.data.delta:
                full_text += event.data.delta
    
    print("[응답]")
    print(full_text)
    print("\n[이벤트 통계]")
    for event_type, count in event_counts.items():
        print(f"  {event_type}: {count}회")

await analyze_stream_events()

=== 스트림 이벤트 분석 ===
스트림 이벤트 수집 중...

[응답]
AI의 미래는 인간과의 협력을 통해 다양한 분야에서 혁신과 효율성을 극대화하는 방향으로 발전할 것입니다.

[이벤트 통계]
  agent_updated_stream_event: 1회
  raw_response_event: 37회
  run_item_stream_event: 1회


### 스트리밍 실행의 특징

장점:
  - 실시간 사용자 피드백 제공
  - 체감 대기 시간 감소
  - 긴 응답에서 사용자 경험 향상
  - 점진적 출력으로 인터랙티브한 느낌

단점:
  - 구현 복잡성 증가
  - 이벤트 처리 로직 필요
  - 전체 응답이 필요한 경우 불편

적합한 사용 사례:
  - 채팅 인터페이스
  - 긴 형식의 콘텐츠 생성
  - 실시간 번역
  - 대화형 스토리텔링
  - 사용자 참여가 중요한 애플리케이션

### 25.4 스트리밍 응답 수집기

실제 애플리케이션에서 활용할 수 있는 스트리밍 응답 수집기 클래스를 구현한다.

In [18]:
class StreamingResponseCollector:
    """
    스트리밍 응답을 수집하고 분석하는 클래스이다.
    """
    
    def __init__(self):
        self.full_text = ""
        self.chunks = []
        self.event_count = 0
    
    async def collect_stream(self, result, on_chunk=None):
        """
        스트림을 수집한다.
        
        Args:
            result: 스트리밍 결과 객체
            on_chunk: 각 청크마다 호출할 콜백 함수
        """
        async for event in result.stream_events():
            self.event_count += 1
            
            if event.type == "raw_response_event":
                if hasattr(event.data, 'delta') and event.data.delta:
                    chunk = event.data.delta
                    self.chunks.append(chunk)
                    self.full_text += chunk
                    
                    # 콜백 호출
                    if on_chunk:
                        on_chunk(chunk)
    
    def get_statistics(self):
        """수집 통계를 반환한다."""
        return {
            "total_events": self.event_count,
            "total_chunks": len(self.chunks),
            "total_characters": len(self.full_text),
            "average_chunk_size": len(self.full_text) / max(len(self.chunks), 1)
        }

print("StreamingResponseCollector 클래스가 정의되었다.")

StreamingResponseCollector 클래스가 정의되었다.


In [19]:
# 스트리밍 수집기 테스트
async def test_collector():
    agent = Agent(
        name="TestAgent",
        instructions="도움이 되는 응답을 제공하세요."
    )
    
    collector = StreamingResponseCollector()
    result = Runner.run_streamed(agent, "Python의 주요 특징 3가지를 설명해주세요.")
    
    print("=== 스트리밍 수집기 테스트 ===")
    print("\n[실시간 출력]")
    
    # 실시간 출력 콜백
    await collector.collect_stream(
        result, 
        on_chunk=lambda chunk: print(chunk, end="", flush=True)
    )
    
    print("\n\n[수집 통계]")
    stats = collector.get_statistics()
    for key, value in stats.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.2f}")
        else:
            print(f"  {key}: {value}")

await test_collector()

=== 스트리밍 수집기 테스트 ===

[실시간 출력]
네, Python의 주요 특징 3가지는 다음과 같습니다.

1. **간결하고 읽기 쉬운 문법**  
   Python은 코드가 간단하고 직관적으로 작성됩니다. 들여쓰기를 이용해 코드의 블록을 구분하므로 가독성이 높아 초보자도 쉽게 배울 수 있습니다.

2. **높은 생산성**  
   풍부한 표준 라이브러리와 다양한 외부 패키지를 통해 복잡한 기능도 적은 코드로 빠르게 개발할 수 있습니다. 덕분에 개발 시간이 단축되고 생산성이 높아집니다.

3. **운영체제 및 플랫폼 독립성**  
   Python은 윈도우, macOS, 리눅스 등 다양한 운영체제에서 동일한 코드로 실행할 수 있습니다. 별도의 수정 없이 여러 환경에서 개발이 가능합니다.

필요에 따라 추가 설명이 필요하거나 다른 특징이 궁금하다면 말씀해주세요!

[수집 통계]
  total_events: 212
  total_chunks: 202
  total_characters: 403
  average_chunk_size: 2.00


---

## 실행 패턴 비교 요약

세 가지 실행 패턴을 비교하여 어떤 상황에서 어떤 패턴을 사용해야 하는지 정리한다.

In [20]:
print("="*60)
print("실행 패턴 비교 요약")
print("="*60)
print()
print(f"{'패턴':<15} {'메서드':<25} {'사용 사례':<30}")
print("-"*70)
print(f"{'동기':<15} {'Runner.run_sync()':<25} {'스크립트, 배치 처리, CLI':<30}")
print(f"{'비동기':<15} {'Runner.run()':<25} {'웹 서버, 대량 처리, API':<30}")
print(f"{'스트리밍':<15} {'Runner.run_streamed()':<25} {'채팅 UI, 실시간 피드백':<30}")
print()
print("선택 가이드:")
print("  - 단일 쿼리, 간단한 스크립트 -> 동기 실행")
print("  - 다중 쿼리, 웹 서버 -> 비동기 실행")
print("  - 사용자 인터페이스, 실시간 피드백 -> 스트리밍 실행")
print()
print("성능 특성:")
print("  - 동기: 가장 단순, 순차적 처리")
print("  - 비동기: 동시 처리로 처리량 향상")
print("  - 스트리밍: 체감 지연 시간 최소화")

실행 패턴 비교 요약

패턴              메서드                       사용 사례                         
----------------------------------------------------------------------
동기              Runner.run_sync()         스크립트, 배치 처리, CLI              
비동기             Runner.run()              웹 서버, 대량 처리, API              
스트리밍            Runner.run_streamed()     채팅 UI, 실시간 피드백                

선택 가이드:
  - 단일 쿼리, 간단한 스크립트 -> 동기 실행
  - 다중 쿼리, 웹 서버 -> 비동기 실행
  - 사용자 인터페이스, 실시간 피드백 -> 스트리밍 실행

성능 특성:
  - 동기: 가장 단순, 순차적 처리
  - 비동기: 동시 처리로 처리량 향상
  - 스트리밍: 체감 지연 시간 최소화


---

## 마무리

이 튜토리얼에서는 OpenAI Agents SDK의 세 가지 실행 패턴을 다루었다. 다음과 같은 내용을 학습하였다:

1. **동기 실행 (Synchronous)**: `Runner.run_sync()`를 사용한 간단한 블로킹 실행, 스크립트와 배치 처리에 적합
2. **비동기 실행 (Asynchronous)**: `Runner.run()`과 `asyncio.gather()`를 사용한 동시 처리, 웹 서버와 대량 처리에 적합
3. **스트리밍 실행 (Streaming)**: `Runner.run_streamed()`를 사용한 실시간 출력, 채팅 인터페이스에 적합

각 패턴의 장단점을 이해하고 사용 사례에 맞는 패턴을 선택하면, 성능과 사용자 경험 모두를 최적화할 수 있다.